In [39]:
#import packages and functions
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import fiona

import shapely
from shapely.validation import make_valid


In [40]:
from general_functions import *

In [41]:
pd.set_option('display.max_columns', None)

In [42]:
#define relative paths
waterschap = 'Scheldestromen'
path_Scheldestromen = '..\..\Data_preprocessed\Waterschappen\Scheldestromen\Scheldestromen.gpkg'
output_gpkg_path = "../../Data_postprocessed/Waterschappen/Scheldestromen"


In [43]:
Scheldestromen = read_gpkg_layers(gpkg_path = path_Scheldestromen, 
                                  variables = ['stuw', 
                                               'gemaal', 
                                               # 'afsluitmiddel',
                                               'duikersifonhevel',
                                               # 'hydroobject',
                                               # 'peilgebiedvigerend',
                                               # 'peilgebiedpraktijk',
                                               # 'peilafwijkinggebied',
                                               # 'streefpeil',
                                              ],
                                 engine = 'pyogrio') 
                            

In [44]:
# #the peilgebieden and streefpeilen do not contain overlapping values. Scheldestromen has delivered additional files as shapes
Scheldestromen['peilgebied'] = gpd.read_file('..\..\Data_preprocessed\Waterschappen\Scheldestromen\Praktijkpeilgebieden_20231204.shp')

In [45]:
#Third nalevering
Scheldestromen_nalevering = read_gpkg_layers(gpkg_path = r"D:\Users\Bruijns\Documents\PR4750_20\Data_preprocessed\Waterschappen\Scheldestromen\Oplevering_Scheldestromen_20240328\Oplevering_20240328.gdb", 
                                             variables = ['Gemalen', 
                                                          'main_Hydroobject_regionaal', 
                                                          'main_Peilgebiedpraktijk_regionaal'],
                                             engine = 'pyogrio') 

In [46]:
Scheldestromen_nalevering['main_Hydroobject_regionaal'] = Scheldestromen_nalevering['main_Hydroobject_regionaal'].loc[Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal != ('None')]
Scheldestromen_nalevering['main_Hydroobject_regionaal'] = Scheldestromen_nalevering['main_Hydroobject_regionaal'].loc[Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal != ('')]
Scheldestromen_nalevering['main_Hydroobject_regionaal'] = Scheldestromen_nalevering['main_Hydroobject_regionaal'].loc[Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal != ('Koppeling afvoer (extremen)')]
Scheldestromen_nalevering['main_Hydroobject_regionaal'] = Scheldestromen_nalevering['main_Hydroobject_regionaal'].loc[Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal != ('Koppeling afvoer (winter)')]
Scheldestromen_nalevering['main_Hydroobject_regionaal'] = Scheldestromen_nalevering['main_Hydroobject_regionaal'].dropna(subset='regionaal')
Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal.unique()


array(['Ja', 'Koppeling afvoer (energiebesparing)',
       'Ja (+ aanvoertraject)', 'Koppeling aanvoer',
       'Koppeling afvoer (zomer)'], dtype=object)

In [47]:
test =[Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal.str.contains('Ja|Koppeling afvoer (zomer)|Koppeling aanvoer|Koppeling afvoer (energiebesparing)')]

C:\Users\Bruijns\AppData\Local\Temp\ipykernel_18924\103704338.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test =[Scheldestromen_nalevering['main_Hydroobject_regionaal'].regionaal.str.contains('Ja|Koppeling afvoer (zomer)|Koppeling aanvoer|Koppeling afvoer (energiebesparing)')]


In [48]:
Scheldestromen['gemaal'] = Scheldestromen_nalevering['Gemalen']
Scheldestromen['hydroobject'] = Scheldestromen_nalevering['main_Hydroobject_regionaal']
Scheldestromen['peilgebied'] = pd.merge(left = Scheldestromen_nalevering['main_Peilgebiedpraktijk_regionaal'],
                                                   right = Scheldestromen['peilgebied'],
                                                   left_on = 'naam',
                                                   right_on = 'GPGIDENT',
                                                   how = 'left',
                                                   suffixes = ('', '_merged_oudelevering'))

In [49]:
Scheldestromen['aggregation_area'] = Scheldestromen['peilgebied'].dissolve(by='RWE_ID')#.plot()
Scheldestromen['aggregation_area'] = Scheldestromen['aggregation_area'][['GPGIDENT', 'nen3610id', 'GLOBALID', 'geometry']]
Scheldestromen['aggregation_area'].rename(columns={'GPGIDENT':'code',
                                                   'GLOBALID': 'globalid'}, 
                                          inplace=True)

In [51]:
#determine aanvoer en afvoer gemalen
Scheldestromen['gemaal']['func_aanvoer'], Scheldestromen['gemaal']['func_afvoer'], Scheldestromen['gemaal']['func_circulatie']  = False, False, False #default is False
Scheldestromen['gemaal']['functiegemaal'] = Scheldestromen['gemaal']['KGMFUNCT'].astype(str) 

Scheldestromen['gemaal'].loc[Scheldestromen['gemaal'].functiegemaal.astype(str).str.contains('8'), 'func_circulatie'] = True
Scheldestromen['gemaal'].loc[Scheldestromen['gemaal'].functiegemaal.astype(str).str.contains('2|4|6|7|98|99'), 'func_afvoer'] = True
Scheldestromen['gemaal'].loc[Scheldestromen['gemaal'].functiegemaal.astype(str).str.contains('1|3'), 'func_aanvoer'] = True
# Scheldestromen['gemaal'].loc[(Scheldestromen['gemaal'].func_afvoer == False) &
#                        (Scheldestromen['gemaal'].func_aanvoer == False) &
#                        (Scheldestromen['gemaal'].func_circulatie == False), 'func_afvoer'] = True #set to afvoergemaal is there the function is unknown

In [52]:
print(len(Scheldestromen['peilgebied'].code.unique()))
print(len(Scheldestromen['peilgebied'].code))

1
852


In [53]:
Scheldestromen['peilgebied']['code'] = Scheldestromen['peilgebied']['GPGIDENT']
Scheldestromen['peilgebied'] = Scheldestromen['peilgebied'].reset_index(drop=True)
Scheldestromen['peilgebied']['code'] = Scheldestromen['peilgebied']['code'].astype(str) + '_dummy_id_' + Scheldestromen['peilgebied'].index.astype(str)

Scheldestromen['peilgebied']['globalid'] = 'dummy_globalid_peilgebied_' + Scheldestromen['peilgebied'].index.astype(str)

Scheldestromen['peilgebied']['waterhoogte'] = Scheldestromen['peilgebied']['GPGZP'] 
Scheldestromen['peilgebied']['nen3610id'] = 'dummy_nen3610id_peilgebied_' + Scheldestromen['peilgebied'].index.astype(str)

Scheldestromen['streefpeil'] = gpd.GeoDataFrame()
Scheldestromen['streefpeil']['waterhoogte'] = Scheldestromen['peilgebied']['waterhoogte']
Scheldestromen['streefpeil']['globalid'] = Scheldestromen['peilgebied']['globalid']
Scheldestromen['streefpeil']['geometry'] = np.nan


Scheldestromen['peilgebied'] = Scheldestromen['peilgebied'][['code', 'nen3610id', 'globalid', 'geometry']]

C:\Users\Bruijns\AppData\Local\Temp\ipykernel_18924\3963481977.py:13: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  Scheldestromen['streefpeil']['geometry'] = np.nan


In [54]:
#convert multiz points to points
Scheldestromen['stuw'].geometry = Scheldestromen['stuw'].centroid
Scheldestromen['gemaal'].geometry = Scheldestromen['gemaal'].centroid


# Scheldestromen

In [55]:
#stuw
Scheldestromen['stuw'] = Scheldestromen['stuw'][['code', 'nen3610id', 'globalid', 'geometry']]

#gemaal
Scheldestromen['gemaal'] = Scheldestromen['gemaal'][['KGMNAAM', 'KGM_GPGIN', 'GLOBALID', 'func_afvoer', 'func_aanvoer', 'func_circulatie', 'geometry']]
Scheldestromen['gemaal'].rename(columns={'KGMNAAM':'code',
                                         'GLOBALID': 'globalid'}, 
                                inplace=True)

#hydroobject
Scheldestromen['hydroobject']['code'] = Scheldestromen['hydroobject']['naam']
Scheldestromen['hydroobject'] = Scheldestromen['hydroobject'][['code', 'nen3610id', 'globalid', 'geometry']]

#are filled in for 99%, but not 100%:
Scheldestromen['hydroobject']['code'] = 'dummy_code_hydroobject_' + Scheldestromen['hydroobject'].index.astype(str)
Scheldestromen['hydroobject']['nen3610id'] = 'dummy_nen3610id_hydroobject_' + Scheldestromen['hydroobject'].index.astype(str)
Scheldestromen['hydroobject']['globalid'] = 'dummy_globalid_hydroobject_' + Scheldestromen['hydroobject'].index.astype(str)

#duikersifonhevel
Scheldestromen['duikersifonhevel'] = Scheldestromen['duikersifonhevel'][['code', 'nen3610id', 'globalid', 'geometry']]

#add the duikersifonhevels to the hydroobjecten
Scheldestromen['hydroobject'] = pd.concat([Scheldestromen['hydroobject'], Scheldestromen['duikersifonhevel']])
Scheldestromen['hydroobject'] = gpd.GeoDataFrame(Scheldestromen['hydroobject'], geometry = 'geometry')

C:\Users\Bruijns\AppData\Local\Temp\ipykernel_18924\429676109.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Scheldestromen['gemaal'].rename(columns={'KGMNAAM':'code',
C:\Users\Bruijns\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Bruijns\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [56]:
# pd.merge(left = Scheldestromen['peilgebiedpraktijk'],
#          right = Scheldestromen['streefpeil'],
#          left_on = 'globalid',
#          right_on = 'peilgebiedpraktijkid')

In [57]:
Scheldestromen['stuw'] = Scheldestromen['stuw'][['code', 'nen3610id', 'globalid', 'geometry']]
# Scheldestromen['gemaal'] = Scheldestromen['gemaal'][['code', 'nen3610id', 'globalid', 'geometry']]
Scheldestromen['hydroobject'] = Scheldestromen['hydroobject'][['code', 'nen3610id', 'globalid', 'geometry']]
Scheldestromen['duikersifonhevel'] = Scheldestromen['duikersifonhevel'][['code', 'nen3610id', 'globalid', 'geometry']]


In [58]:
# Scheldestromen['peilgebiedpraktijk']['geometry'] = Scheldestromen['peilgebiedpraktijk'].buffer(distance = 0)
# Scheldestromen['peilafwijkinggebied']['geometry'] = Scheldestromen['peilafwijkinggebied'].buffer(distance = 0)

In [59]:
# peilgebied = pd.merge(left = Scheldestromen['streefpeil'],
#                       right = Scheldestromen['peilgebiedpraktijk'],
#                       left_on = 'peilgebiedpraktijkid',
#                       right_on = 'globalid')

In [60]:
# streefpeil = gpd.GeoDataFrame()
# streefpeil['waterhoogte'] = peilgebied['waterhoogte']
# streefpeil['globalid'] = peilgebied['peilgebiedpraktijkid']
# streefpeil['geometry'] = np.nan
# Scheldestromen['streefpeil'] = gpd.GeoDataFrame(streefpeil)

In [61]:
# Scheldestromen['peilgebied'] = gpd.GeoDataFrame()
# Scheldestromen['peilgebied'][['code', 'nen3610id', 'globalid', 'geometry']] = peilgebied[['code', 'nen3610id_y', 'globalid_y', 'geometry_y']]

In [62]:
#delete irrelvant data
variables = ['peilgebiedpraktijk', 'peilgebiedvigerend', 'peilafwijkinggebied']

for variable in variables:
    if str(variable) in Scheldestromen:
        del Scheldestromen[variable]

In [63]:
Scheldestromen['streefpeil'].waterhoogte = Scheldestromen['streefpeil'].waterhoogte.round(2)

### Check for the correct keys and columns

In [64]:
show_layers_and_columns(waterschap = Scheldestromen)

stuw
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

gemaal
['code' 'KGM_GPGIN' 'globalid' 'func_afvoer' 'func_aanvoer'
 'func_circulatie' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

duikersifonhevel
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

peilgebied
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

hydroobject
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

aggregation_area
['code' 'nen3610id' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  EPSG:28992

streefpeil
['waterhoogte' 'globalid' 'geometry']
type =  <class 'geopandas.geodataframe.GeoDataFrame'>
crs =  None



In [65]:
# for i in range(len(Scheldestromen['peilgebied'])):    
#     if type(Scheldestromen['peilgebied'].loc[i, 'geometry']) == Polygon:
#         Scheldestromen['peilgebied'].loc[i, 'geometry'].plot()

In [66]:
# merged = pd.merge(left = Scheldestromen['peilgebied'],
#          right = Scheldestromen['streefpeil'],
#          on = 'globalid')

# merged[merged.waterhoogte.isna()]


### Store data

In [67]:
# Check if the directory exists
if not os.path.exists(output_gpkg_path):
    # If it doesn't exist, create it
    os.makedirs(output_gpkg_path)
    
store_data(waterschap = Scheldestromen, 
           output_gpkg_path = output_gpkg_path + '/Scheldestromen')
